In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# create a sample pandas dataframe
df = pd.DataFrame({
    'Gender': ['M', 'F', 'M', 'F', 'M', 'F'],
    'City': ['New York', 'San Francisco', 'New York', 'San Francisco', 'New York', 'San Francisco'],
    'Education': ['High School', 'College', 'College', 'Graduate', 'High School', 'Graduate'],
    'Age': [25, 30, 35, 40, 45, 50],
    'Income': [50000, 60000, 70000, 80000, 90000, 100000],
    'Target': [0, 1, 0, 1, 0, 1]
})

# create OneHotEncoder object
ohe = OneHotEncoder(sparse=False)

# fit and transform the categorical features
ohe_features = ohe.fit_transform(df[['Gender', 'City', 'Education']])

# create new dataframe with one-hot encoded categorical features
ohe_df = pd.concat([df.drop(['Gender', 'City', 'Education'], axis=1), pd.DataFrame(ohe_features)], axis=1)

# print the original and one-hot encoded dataframes
print('Original Dataframe:')
print(df)
print('One-Hot Encoded Dataframe:')
print(ohe_df)


Original Dataframe:
  Gender           City    Education  Age  Income  Target
0      M       New York  High School   25   50000       0
1      F  San Francisco      College   30   60000       1
2      M       New York      College   35   70000       0
3      F  San Francisco     Graduate   40   80000       1
4      M       New York  High School   45   90000       0
5      F  San Francisco     Graduate   50  100000       1
One-Hot Encoded Dataframe:
   Age  Income  Target    0    1    2    3    4    5    6
0   25   50000       0  0.0  1.0  1.0  0.0  0.0  0.0  1.0
1   30   60000       1  1.0  0.0  0.0  1.0  1.0  0.0  0.0
2   35   70000       0  0.0  1.0  1.0  0.0  1.0  0.0  0.0
3   40   80000       1  1.0  0.0  0.0  1.0  0.0  1.0  0.0
4   45   90000       0  0.0  1.0  1.0  0.0  0.0  0.0  1.0
5   50  100000       1  1.0  0.0  0.0  1.0  0.0  1.0  0.0


In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf

# create a sample dataframe with categorical and numerical columns
data = {'Gender': ['Male', 'Female', 'Female', 'Male', 'Other'],
        'Age': [25, 30, 18, 35, 42],
        'Income': [50000, 80000, 12000, 100000, 60000],
        'Education': ['High School', 'College', 'College', 'Graduate','Graduate'],
        'Married': [True, False, False, True, False]}
df = pd.DataFrame(data)

# convert the categorical column to string
df['Education'] = df['Education'].astype(str)

# one-hot encode the categorical columns
cat_cols = ['Gender', 'Education']
num_cols = ['Age', 'Income', 'Married']

ohe = OneHotEncoder()
ohe.fit(df[cat_cols])

# scale the numerical columns
scaler = StandardScaler()
scaler.fit(df[num_cols])

# create a feature layer using the encoded categorical columns and scaled numerical columns
feature_columns = []
for col in cat_cols:
    feature_columns.append(tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(col, vocabulary_list=df[col].unique().tolist())))

for col in num_cols:
    feature_columns.append(tf.feature_column.numeric_column(col))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

# create a tensorflow dataset
labels = df.pop('Income')
dataset = tf.data.Dataset.from_tensor_slices((dict(df), labels))
dataset = dataset.shuffle(buffer_size=len(df))
dataset = dataset.batch(batch_size=32)

# apply the feature layer to the dataset
dataset = dataset.map(lambda x, y: (feature_layer(x), y))


ValueError: in user code:

    File "/tmp/ipykernel_1014/2206681222.py", line 45, in None  *
        lambda x, y: (feature_layer(x), y)
    File "/root/miniconda3/envs/tfdml_plugin/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "dense_features_2" "                 f"(type DenseFeatures).
    
    Feature Income is not in features dictionary.
    
    Call arguments received by layer "dense_features_2" "                 f"(type DenseFeatures):
      • features={'Gender': 'tf.Tensor(shape=(None,), dtype=string)', 'Age': 'tf.Tensor(shape=(None,), dtype=int64)', 'Education': 'tf.Tensor(shape=(None,), dtype=string)', 'Married': 'tf.Tensor(shape=(None,), dtype=bool)'}
      • cols_to_output_tensors=None
      • training=False
